# zorang2 Code


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-vdqy9toe because the default path (/home/sldev1/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2023-01-13 13:56:08.017515: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sldev1/anaconda3/envs/dms2/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.0/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-01-13 13:56:08.017535: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
# filepath = filepath[:4]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

['/media/sldev1/T7/DMS2/01. 김형민_1201.mp4', '/media/sldev1/T7/DMS2/02. 김두용_1205.mp4', '/media/sldev1/T7/DMS2/03. 주용현_1206.mp4', '/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4', '/media/sldev1/T7/DMS2/05. 이민정_1209.mp4', '/media/sldev1/T7/DMS2/06. 김세희_1213.mp4', '/media/sldev1/T7/DMS2/07. 이선진_1213.mp4', '/media/sldev1/T7/DMS2/08. 정승아_1214.mp4', '/media/sldev1/T7/DMS2/09. 류채원_1214.mp4', '/media/sldev1/T7/DMS2/10. 최서영_1214.mp4', '/media/sldev1/T7/DMS2/11. 하현옥_1214.mp4', '/media/sldev1/T7/DMS2/12. 윤호영_1214.mp4', '/media/sldev1/T7/DMS2/13. 김설양_1215.mp4', '/media/sldev1/T7/DMS2/14. 박세익_1215.mp4', '/media/sldev1/T7/DMS2/15. 김주일_1215.mp4', '/media/sldev1/T7/DMS2/16. 유현지_1215.mp4', '/media/sldev1/T7/DMS2/17. 임우준_1216.mp4', '/media/sldev1/T7/DMS2/18. 송영달_1216.mp4', '/media/sldev1/T7/DMS2/19. 표성민_1216.mp4', '/media/sldev1/T7/DMS2/20. 박동수_1216.mp4', '/media/sldev1/T7/DMS2/21. 유은식_1216.mp4', '/media/sldev1/T7/DMS2/22. 안만석_1219.mp4', '/media/sldev1/T7/DMS2/23. 최시리_1219.mp4', '/media/sldev1/T7/DMS2/24. 서창희_12

In [ ]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

  0%|                                                                                          | 0/50 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/media/sldev1/T7/DMS2/01. 김형민_1201.mp4


  2%|█▌                                                                           | 1/50 [52:49<43:08:13, 3169.26s/it]

/media/sldev1/T7/DMS2/02. 김두용_1205.mp4
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion faile

...

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion faile

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion faile

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion faile

  4%|███                                                                        | 2/50 [1:51:14<44:53:35, 3366.98s/it]

/media/sldev1/T7/DMS2/03. 주용현_1206.mp4
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

